In [9]:
import os

print("CPU 코어 수:", os.cpu_count())


CPU 코어 수: 32


In [2]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 3.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from scipy.interpolate import griddata
from rasterio.transform import from_origin
import rasterio
import multiprocessing
import rasterio.mask
import tempfile
import os

# 1. 도로 및 등고선 데이터 불러오기
roads = gpd.read_file("인도_조도.shp")
contours = gpd.read_file("N3L_F0010000_N.shp")

# 좌표계 통일 (EPSG:5186)
roads = roads.to_crs(epsg=5186)
contours = contours.to_crs(epsg=5186)

# 2. 인천 경계 추출 및 등고선 자르기
incheon_boundary = roads[roads['sido_nm'] == '인천광역시'].geometry.union_all()
incheon_gdf = gpd.GeoDataFrame(geometry=[incheon_boundary], crs=roads.crs)
contours_incheon = gpd.overlay(contours, incheon_gdf, how='intersection')

# 3. 고도 보간을 위한 그리드 생성
x = contours_incheon.geometry.centroid.x.values
y = contours_incheon.geometry.centroid.y.values
z = contours_incheon['CONT'].astype(float).values

grid_res = 10  # 해상도 10m
xmin, ymin, xmax, ymax = contours_incheon.total_bounds
grid_x, grid_y = np.meshgrid(
    np.arange(xmin, xmax, grid_res),
    np.arange(ymin, ymax, grid_res)
)
grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')

# 4. 경사도 계산
dy, dx = np.gradient(grid_z, grid_res)
slope = np.sqrt(dx**2 + dy**2)
transform = from_origin(xmin, ymax, grid_res, grid_res)

# 5. 경사도 래스터 임시 저장 (파일 저장은 하지만 사용 후 삭제 예정)
with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmpfile:
    temp_slope_path = tmpfile.name

with rasterio.open(
    temp_slope_path, 'w',
    driver='GTiff',
    height=slope.shape[0],
    width=slope.shape[1],
    count=1,
    dtype='float32',
    transform=transform,
    crs=roads.crs
) as dst:
    dst.write(slope.astype('float32'), 1)

# 6. 병렬 처리 함수 정의
def compute_mean_slope(geom):
    try:
        with rasterio.open(temp_slope_path) as src:
            out_image, out_transform = rasterio.mask.mask(src, [geom], crop=True)
            data = out_image[0]
            masked = np.ma.masked_array(data, data == src.nodata)
            return masked.mean()
    except Exception:
        return np.nan

# 7. 병렬 처리 실행
with rasterio.open(temp_slope_path) as src:
    nodata_val = src.nodata

with multiprocessing.Pool(processes=os.cpu_count()) as pool:
    mean_slopes = pool.map(compute_mean_slope, roads.geometry)

roads["mean_slope"] = mean_slopes


# 8. 결과 출력
print(roads[["objectid", "mean_slope"]].head())

# 9. 임시파일 삭제
os.remove(temp_slope_path)


   objectid  mean_slope
0         1    0.001901
1         2    0.000877
2         3    0.001955
3         4    0.000745
4         5    0.003422


In [15]:
import numpy as np

roads['mean_slope_percent'] = roads['mean_slope'] * 100
roads['mean_slope_degree'] = np.degrees(np.arctan(roads['mean_slope']))

print(roads[['mean_slope', 'mean_slope_percent', 'mean_slope_degree']].head())


   mean_slope  mean_slope_percent  mean_slope_degree
0    0.001901            0.190101           0.108919
1    0.000877            0.087739           0.050271
2    0.001955            0.195532           0.112031
3    0.000745            0.074522           0.042698
4    0.003422            0.342181           0.196055


In [16]:
roads['mean_slope_degree'].value_counts()

mean_slope_degree
0.000000e+00    3518
7.971641e-03      20
9.246231e-02      12
1.594328e-02      10
1.272222e-16      10
                ... 
1.699170e-02       1
1.903070e-02       1
1.349394e-02       1
9.653630e-02       1
6.796352e-02       1
Name: count, Length: 9132, dtype: int64

In [18]:
roads.to_file("인도_조도_경사도.shp")

/tmp/ipykernel_1260/1081746634.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  roads.to_file("인도_조도_경사도.shp")
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'mean_slope_percent' to 'mean_slo_1'
  ogr_write(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'mean_slope_degree' to 'mean_slo_2'
  ogr_write(


In [1]:
import pandas as pd
df_1=pd.read_csv("인천광역시_조도_202110.csv",encoding='cp949')
df_2=pd.read_csv("인천광역시_조도_202111.csv",encoding='cp949')

In [2]:
# 두 개의 센서 데이터를 합치기
df = pd.concat([df_1, df_2], ignore_index=True)

In [3]:
df

,센서명,시간,위도,경도,지오해시,시,군구,동,조도
0,M-IOT-00000007,2021-10-01 00:00:00,37.494364,126.726780,wydj79xs,인천광역시,부평구,부평동,13
1,M-IOT-00000007,2021-10-01 00:00:01,37.494364,126.727123,wydj79xu,인천광역시,부평구,부평동,3
2,M-IOT-00000007,2021-10-01 00:00:02,37.494364,126.727123,wydj79xu,인천광역시,부평구,부평동,3
3,M-IOT-00000007,2021-10-01 00:00:03,37.494364,126.727123,wydj79xu,인천광역시,부평구,부평동,3
4,M-IOT-00000007,2021-10-01 00:00:04,37.494192,126.727467,wydj7c85,인천광역시,부평구,부평동,3
...,...,...,...,...,...,...,...,...,...
61867461,M-IOT-000000D8,2021-11-14 11:08:48,37.413683,126.624126,wydhc6uy,인천광역시,연수구,송도동,12422
61867462,M-IOT-000000D8,2021-11-14 11:08:49,37.413683,126.624126,wydhc6uy,인천광역시,연수구,송도동,12422
61867463,M-IOT-000000D8,2021-11-14 11:08:50,37.413855,126.624126,wydhc6uz,인천광역시,연수구,송도동,12422
61867464,M-IOT-000000D8,2021-11-14 11:08:51,37.413855,126.624126,wydhc6uz,인천광역시,연수구,송도동,12422


In [4]:
import geopandas as gpd
import pandas as pd

# 1. Shapefile 읽기
gdf = gpd.read_file('인도.shp')

# 2. 좌표계 설정 (없거나 잘못된 경우)
if gdf.crs is None:
    print("좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.")
    gdf.set_crs(epsg=5186, inplace=True)
elif gdf.crs.to_epsg() != 5186:
    print(f"현재 좌표계: {gdf.crs}")
    print("좌표계를 WKID 5186으로 재지정합니다.")
    gdf.to_crs(epsg=5186, inplace=True)

# 3. 좌표계 변환 (TM -> WGS84)
gdf = gdf.to_crs(epsg=4326)

# 4. 중심점 계산
gdf['rep_point'] = gdf.geometry.representative_point()
gdf[['위도', '경도']] = gdf['rep_point'].apply(lambda p: pd.Series([p.y, p.x]))

# 결과 확인
print(gdf[['위도', '경도']].head())


좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.
          위도          경도
0  37.475196  126.685533
1  37.474532  126.685289
2  37.471580  126.687159
3  37.472759  126.685074
4  37.473605  126.683281


In [31]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, GeometryCollection  # Point는 여기서 가져옴
import numpy as np

# 벡터화 (Shapely 2.x)
df['geometry'] = gpd.points_from_xy(df['경도'], df['위도'])
geo_df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

print(geo_df.head())


              센서명                   시간         위도          경도      지오해시  \
0  M-IOT-00000007  2021-10-01 00:00:00  37.494364  126.726780  wydj79xs   
1  M-IOT-00000007  2021-10-01 00:00:01  37.494364  126.727123  wydj79xu   
2  M-IOT-00000007  2021-10-01 00:00:02  37.494364  126.727123  wydj79xu   
3  M-IOT-00000007  2021-10-01 00:00:03  37.494364  126.727123  wydj79xu   
4  M-IOT-00000007  2021-10-01 00:00:04  37.494192  126.727467  wydj7c85   

       시   군구    동  조도                    geometry  
0  인천광역시  부평구  부평동  13  POINT (126.72678 37.49436)  
1  인천광역시  부평구  부평동   3  POINT (126.72712 37.49436)  
2  인천광역시  부평구  부평동   3  POINT (126.72712 37.49436)  
3  인천광역시  부평구  부평동   3  POINT (126.72712 37.49436)  
4  인천광역시  부평구  부평동   3  POINT (126.72747 37.49419)  


In [11]:
import pandas as pd
import geopandas as gpd

df['geometry'] = gpd.points_from_xy(df['경도'], df['위도'])
geo_df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

gdf['시간'] = None
gdf['조도'] = None

for emd_nm in gdf['sgg_nm'].unique():
    gdf_sub = gdf[gdf['sgg_nm'] == emd_nm]
    df_sub = geo_df[geo_df['군구'] == emd_nm]

    if df_sub.empty:
        print(f"{emd_nm}: df_sub에 데이터 없음, 건너뜀")
        continue

    if gdf_sub.crs != df_sub.crs:
        df_sub = df_sub.to_crs(gdf_sub.crs)

    nearest = gpd.sjoin_nearest(
        gdf_sub, 
        df_sub[['시간', '조도', 'geometry']], 
        how='left', 
        max_distance=0.01,
        lsuffix='_gdf', 
        rsuffix='_df'
    )

    gdf.loc[nearest.index, '시간'] = nearest['시간__df']
    gdf.loc[nearest.index, '조도'] = nearest['조도__df']

gdf


/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geogr

,objectid,indo_id,sido_nm,sgg_nm,emd_nm,edit_date,edit_code,st_area_sh,st_length_,geometry,rep_point,위도,경도,시간,조도
0,1,544.0,인천광역시,서구,가좌3동,2021-12-03,A,231.591857,123.258680,"POLYGON ((126.68575 37.47503, 126.68574 37.475...",POINT (126.68553 37.4752),37.475196,126.685533,2021-11-17 14:15:41,22786
1,2,542.0,인천광역시,서구,가좌3동,2021-12-03,A,300.798851,330.778248,"POLYGON ((126.68451 37.47387, 126.68451 37.473...",POINT (126.68529 37.47453),37.474532,126.685289,2021-11-17 14:15:41,22786
2,3,525.0,인천광역시,서구,가좌3동,2021-12-03,A,254.342128,153.750610,"POLYGON ((126.68738 37.47145, 126.68737 37.471...",POINT (126.68716 37.47158),37.471580,126.687159,2021-10-19 11:24:55,3903
3,4,526.0,인천광역시,서구,가좌3동,2021-12-03,A,1160.365578,666.283231,"POLYGON ((126.68654 37.47196, 126.68658 37.471...",POINT (126.68507 37.47276),37.472759,126.685074,2021-11-03 19:21:30,8
4,5,527.0,인천광역시,서구,가좌3동,2021-12-03,A,134.048600,83.401540,"POLYGON ((126.68347 37.47356, 126.68346 37.473...",POINT (126.68328 37.47361),37.473605,126.683281,2021-11-03 19:21:30,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14870,14870,14603.0,인천광역시,동구,화수2동,2021-12-03,A,94.798031,103.609890,"POLYGON ((126.63369 37.48253, 126.63368 37.482...",POINT (126.63354 37.48264),37.482639,126.633540,2021-10-11 19:36:13,4
14871,14872,14821.0,인천광역시,계양구,효성2동,2021-12-03,A,150.809219,93.936749,"POLYGON ((126.70536 37.52682, 126.70536 37.526...",POINT (126.70535 37.52693),37.526930,126.705348,2021-10-23 17:43:12,598
14872,14873,14867.0,인천광역시,계양구,효성2동,2021-12-03,A,663.671810,330.393697,"POLYGON ((126.70066 37.52803, 126.70002 37.527...",POINT (126.70065 37.52816),37.528156,126.700648,2021-10-29 07:48:18,13312
14873,14874,14868.0,인천광역시,계양구,효성2동,2021-12-03,A,641.428650,406.510166,"POLYGON ((126.71562 37.52799, 126.71616 37.527...",POINT (126.71548 37.5283),37.528302,126.715478,2021-10-11 21:23:32,42


In [15]:
gdf = gdf.drop(columns=['rep_point'])

In [16]:
gdf.to_file("인도_조도.shp", encoding="utf-8")

In [26]:
# gdf 'emd_nm' 고유값
gdf_emd_nm = set(gdf['sgg_nm'].unique())
# geo_df '군구' 고유값
geo_df_gungu = set(geo_df['군구'].unique())

# gdf에는 있는데 geo_df에는 없는 값들
only_in_gdf = gdf_emd_nm - geo_df_gungu
print("gdf에는 있는데 geo_df에는 없는 읍면동명 (emd_nm):", only_in_gdf)

# geo_df에는 있는데 gdf에는 없는 값들
only_in_geo_df = geo_df_gungu - gdf_emd_nm
print("geo_df에는 있는데 gdf에는 없는 군구명:", only_in_geo_df)


gdf에는 있는데 geo_df에는 없는 읍면동명 (emd_nm): set()
geo_df에는 있는데 gdf에는 없는 군구명: {'강화군', '옹진군'}


In [4]:
import ee
import geemap
import geopandas as gpd
import pandas as pd

# 초기화
ee.Initialize()

# 1. GeoDataFrame 준비
gdf = gpd.read_file('인도_조도.shp')

# 좌표계 맞추기 (필수)
if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs(epsg=4326)

# Earth Engine FeatureCollection으로 변환
fc = geemap.geopandas_to_ee(gdf)

# 2. 고도 및 경사도 이미지 불러오기
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation)

# 3. 각 폴리곤에 대해 평균 경사도 추출
def get_mean_slope(feature):
    mean_dict = slope.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    )
    return feature.set('mean_slope', mean_dict.get('slope'))

# 계산 적용
slope_fc = fc.map(get_mean_slope)

# 4. 다시 Pandas DataFrame으로 변환
slope_gdf = geemap.ee_to_geopandas(slope_fc)

# 원본 gdf에 병합
gdf['mean_slope'] = slope_gdf['mean_slope']

ModuleNotFoundError: No module named 'StringIO'